# Исследование надежности заемщиков

## Общая информация о данных

In [1]:
import pandas as pd
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [6]:
data['days_employed'] = data['days_employed'].abs()

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [14]:
data['education'] = data['education'].str.lower()

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследуем данные и отвечаем на вопросы

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
def probability(row):
    return str(round((row.sum() / row.count()) * 100, 2)) + '%'

data_children = data.pivot_table(index=['children'], values=['debt'], aggfunc=['count','sum',  probability])
data_children = data_children.sort_values(by=('probability', 'debt'))
print (data_children)

          count   sum probability
           debt  debt        debt
children                         
5             9     0        0.0%
0         14091  1063       7.54%
3           330    27       8.18%
1          4808   444       9.23%
2          2052   194       9.45%
4            41     4       9.76%


**Вывод: При наличие 5 детей просрочки кредита нет, но для объктивной оценки данных мало(всего 9 человек). При отсутствие детей вероятность невозврата кредита низкая - 7.5%, а при наличие детей от 1 до 4 - вероятность невозврата 8-10%.** 

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
data_status = data.pivot_table(index=['family_status'], values=["debt"], aggfunc=['count','sum',  probability])
data_status = data_status.sort_values(by=('probability', 'debt'))
print (data_status)

                       count  sum probability
                        debt debt        debt
family_status                                
вдовец / вдова           951   63       6.62%
в разводе               1189   84       7.06%
женат / замужем        12261  927       7.56%
гражданский брак        4134  385       9.31%
Не женат / не замужем   2796  273       9.76%


**Вывод: Не женатые/не замужние или люди в гражданском браке реже выплачивают кредит в срок, невозврат 9-10%. Люди, побывавшие в браке, реже допускают просрочки.** 

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
data_income = data.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['count','sum',  probability])
data_income = data_income.sort_values(by=('probability', 'debt'))
print (data_income)

                       count   sum probability
                        debt  debt        debt
total_income_category                         
D                        349    21       6.02%
B                       5014   354       7.06%
A                         25     2        8.0%
C                      15921  1353        8.5%
E                         22     2       9.09%


**Вывод: В категории "А","D" и "Е" - людей мало, недостаточное количество для оценки. Люди с категорией "В"(доход 200_001–1_000_000)  лучше возвращают кредит - невозврат 7%, чем люди с категорие "С"(доход 50_001–200_000) - невозврат 8.5%.**  

#### Как разные цели кредита влияют на его возврат в срок?

In [26]:
data_purpose = data.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['count','sum',  probability])
data_purpose = data_purpose.sort_values(by=('probability', 'debt'))
print (data_purpose)

                          count  sum probability
                           debt debt        debt
purpose_category                                
операции с недвижимостью  10751  780       7.26%
проведение свадьбы         2313  183       7.91%
получение образования      3988  369       9.25%
операции с автомобилем     4279  400       9.35%


**Вывод: Лучше всего возвращают кредит при операции с недвижимостью - 7% невозврат, плохо возвращают кредит при операции с автомобилем - невозврат 9.4%.** 

#### Приведите возможные причины появления пропусков в исходных данных.

*Ответ: Причинами, приводящими к появлению пропусков, являются: - невозможность их получения или обработки; - искажение или сокрытие информации; - случайное потеря данный при записи или чтении; - частичное искажение данных (например: отрицательные данные, где должны быть только положительные); - человеческий или технический фактор в результате на вход программ анализа собранных данных поступают неполные сведения или искаженные.* 

#### Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ: Отличие между средним и медианным значением является то, что Среднее - это сумма общих значений в наборе данных, деленная на количество значений, а медиана - это среднее значение набора данных. И поэтому среднее - это среднее значение набора данных, а медиана - это центральное числовое значение набора данных и в нашем случае оптимальное значение.* 

### Общий вывод:

При работе с данными были заполнены пропущенные значения, изменены неверные данные(отрицательные на положительные), удалены дубликаты и выделены категории по доходу и цели кредита. 

Применив сводные таблицы, я оценил, какие критерии влияют на возврат кредита: 
* при отсутствие детей вероятность невозврата кредита низкая - 7.5%, а при наличие детей от 1 до 4 - вероятность невозврата 8-10%; не женатые/не замужние или люди в гражданском браке реже выплачивают кредит в срок, невозврат 9-10%;
* люди с категорией "В"(доход 200_001–1_000_000) лучше возвращают кредит - невозврат 7%, чем люди с категорие "С"(доход 50_001–200_000) - невозврат 8.5%; 
* лучше всего возвращают кредит при операции с недвижимостью - 7% невозврат, плохо возвращают кредит при операции с автомобилем - невозврат 9,4%. 

Вдовец/вдова без детей с доход 200_001–1_000_000 при операции с недвижемостью - это самой оптимальный кредитор.